# L5: Process Inputs: Chaining Prompts

In [ ]:
import openai


def get_completion_from_messages(
    messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500
):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

Implement a complex task with multiple prompts and Extract relevant product and category names

In [ ]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.

Output a python list of objects, where each object has \
the following format:
    'category': <one of Computers and Laptops, \
    Smartphones and Accessories, \
    Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, 
    Audio Equipment, Cameras and Camcorders>,
OR
    'products': <a list of products that must \
    be found in the allowed products below>

Where the categories and products must be found in \
the customer service query.

If a product is mentioned, it must be associated with \
the correct category in the allowed products list below.

If no products or categories are found, output an \
empty list.

Allowed products: 

Computers and Laptops category:
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

Smartphones and Accessories category:
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

Televisions and Home Theater Systems category:
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

Gaming Consoles and Accessories category:
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

Audio Equipment category:
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

Cameras and Camcorders category:
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

Only output the list of objects, with nothing else.
"""

user_message_1 = f"""
 tell me about the smartx pro phone and \
 the fotosnap camera, the dslr one. \
 Also tell me about your tvs """

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": f"{delimiter}{user_message_1}{delimiter}"},
]

category_and_product_response_1 = get_completion_from_messages(messages)

print(category_and_product_response_1)

In [ ]:
user_message_2 = f"""
my router isn't working"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": f"{delimiter}{user_message_2}{delimiter}"},
]

response = get_completion_from_messages(messages)
print(response)

Retrieve detailed product information for extracted products and categories

In [ ]:
# product information
import json

with open("product.json") as f:
    products = json.load(f)


def get_product_by_name(name):
    return products.get(name, None)


def get_products_by_category(category):
    return [product for product in products.values() if product["category"] == category]


print(get_product_by_name("TechPro Ultrabook"))
print(get_products_by_category("Computers and Laptops"))

In [ ]:
print(user_message_1)
print(category_and_product_response_1)

Read Python string into Python list of dictionaries

In [ ]:
def read_string_to_list(input_string):
    if input_string is None:
        return None

    try:
        input_string = input_string.replace(
            "'", '"'
        )  # Replace single quotes with double quotes for valid JSON
        data = json.loads(input_string)
        return data

    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None


category_and_product_list = read_string_to_list(category_and_product_response_1)
print(category_and_product_list)

Retrieve detailed product information for the relevant products and categories

In [ ]:
def generate_output_string(data_list):
    output_string = ""
    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")

            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4) + "\n"

            else:
                print("Error: Invalid object format")

        except Exception as e:
            print(f"Error: {e}")

    return output_string


product_information_for_user_message_1 = generate_output_string(
    category_and_product_list
)
print(product_information_for_user_message_1)

Generate answer to user query based on detailed product information

In [ ]:
system_message = f"""
You are a customer service assistant for a \
large electronic store. \
Respond in a friendly and helpful tone, \
with very concise answers. \
Make sure to ask the user relevant follow up questions.
"""

user_message_1 = f"""
tell me about the smartx pro phone and \
the fotosnap camera, the dslr one. \
Also tell me about your tvs"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message_1},
    {
        "role": "assistant",
        "content": f"""Relevant product information:\n\
 {product_information_for_user_message_1}""",
    },
]

final_response = get_completion_from_messages(messages)
print(final_response)